In [5]:
#encoding:utf8
import pandas as pd
from db import database

db = database.database()

start_date = 20170101
end_date = 20181220

YEAR_LIST = [2015,2016,2017,2018]
MONTH_LIST = [100*x+y for x in YEAR_LIST for y in range(1,13)]

def get_series_by_month(stock_code,yyyymm,freq=30):
    fields = "f_seq,f_date,f_time,f_close,f_high,f_low,f_open,f_preclose,f_amount,f_volume"
    args = [stock_code]
    sql = "select {} from t_mik_stock_{}_{} where f_code=%s".format(fields,str(freq),str(yyyymm))
    items = db.fetch_all(sql,args)

    if len(items)==0:
        df = pd.DataFrame()
    else:
        df = pd.DataFrame(items)
    return df

def get_hq_m(stock_code,start_date,end_date,freq=30):
    items = []
    st_m = start_date/100
    end_m = end_date/100
    m_list = [x for x in MONTH_LIST if x<=end_m and x>=st_m]
    
    #根据月份调取
    for month in m_list:
        df = get_series_by_month(stock_code,month,freq)
        items.append(df)
    df_res = pd.concat(items,axis=0)
    
    #去除开盘数据
    df_res = df_res[~df_res["f_time"].isin([930,1300])]    
    
    df_res["f_datetime"] = df_res["f_date"]*10000 + df_res["f_time"]
    df_res = df_res.sort_values(["f_datetime"])
    df_res = df_res[(df_res["f_date"]>start_date) & (df_res["f_date"]<=end_date)]
    return df_res.reset_index(drop=True)
    
# df_300 = pd.read_excel("000300.xlsx",sheet_name="data")
df_A = pd.read_excel("A_all.xlsx")
# code_300 = df_300["code"].tolist()
code_A = df_A["code"].tolist()


for stock_code in code_A[0:1]:
    print stock_code
    df = get_hq_m(stock_code,start_date,end_date)
    df.to_excel(r"data\30m\{}.xlsx".format(stock_code),sheet_name="data")
    print "stock_code:{} is finished!".format(stock_code)
print "OK!"


['1', '2', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14', '16', '17', '18', '19', '20', '21', '23', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '40', '42', '43', '45', '46', '48', '49', '50', '55', '56', '58', '59', '60', '61', '62', '63', '65', '66', '68', '69', '70', '78', '88', '89', '90', '96', '99', '100', '150', '151', '153', '155', '156', '157', '158', '159', '166', '301', '333', '338', '400', '401', '402', '403', '404', '407', '408', '409', '410', '411', '413', '415', '416', '417', '418', '419', '420', '421', '422', '423', '425', '426', '428', '429', '430', '488', '498', '501', '502', '503', '504', '505', '506', '507', '509', '510', '513', '514', '516', '517', '518', '519', '520', '521', '523', '524', '525', '526', '528', '529', '530', '531', '532', '533', '534', '536', '537', '538', '539', '540', '541', '543', '544', '545', '546', '547', '548', '550', '551', '552', '553', '554', '555', '557', '558', '559', '560', '561', '563', '56

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()


KeyError: 'f_time'

### 30分钟买卖点回测

* 引入包

In [1]:
#encoding:utf8
import pandas as pd
from db import database
from util import indictor

* 读取数据

In [2]:
df = pd.read_excel(r'data\30m\2.xlsx',sheet_name="data")
df

,f_amount,f_close,f_date,f_high,f_low,f_open,f_preclose,f_seq,f_time,f_volume,f_datetime
0,1.102054e+08,20.78,20170103,20.88,20.60,20.63,20.63,12,1000,5311472,201701031000
1,5.633500e+07,20.71,20170103,20.81,20.67,20.79,20.78,13,1030,2716599,201701031030
2,3.060239e+07,20.75,20170103,20.77,20.70,20.72,20.71,14,1100,1475796,201701031100
3,5.279873e+07,20.72,20170103,20.75,20.71,20.75,20.75,15,1130,2546505,201701031130
4,2.963818e+07,20.68,20170103,20.73,20.66,20.72,20.73,17,1330,1432380,201701031330
5,3.242910e+07,20.68,20170103,20.70,20.67,20.68,20.68,18,1400,1567764,201701031400
6,4.392430e+07,20.72,20170103,20.77,20.67,20.69,20.68,19,1430,2119076,201701031430
7,8.691549e+07,20.73,20170103,20.73,20.68,20.73,20.72,20,1500,4197277,201701031500
8,9.080559e+07,20.58,20170104,20.82,20.55,20.76,20.76,30372,1000,4395577,201701041000
9,9.250333e+07,20.53,20170104,20.63,20.45,20.57,20.58,30373,1030,4507435,201701041030


In [3]:
f_close = df["f_close"]
rsi1,rsi2,rsi3 = indictor.RSI(f_close)
df['rsi1'] = rsi1
df["rsi2"] = rsi2
df["L20"] = 20
df["H80"] = 80
df["llv_low"] = indictor.LLV(df["f_low"],3)    #LLV(LOW,3)
df["llv_rsi1"] = indictor.LLV(df["rsi1"],3)    #LLV(RSI1,3)
df["llv_rsi2"] = indictor.LLV(df["rsi2"],6)    #LLV(RSI2,6)

df["cross_rsi1_rsi2"] = indictor.CROSS(df["rsi1"],df["rsi2"])
df["LLV_RSI2_21"] = map(lambda x: 1 if x<21 else 0 ,df["llv_rsi2"])

df["cross_rsi2_20"] = indictor.CROSS(df["rsi2"],df["L20"])
df["LOW_LLV"] = map(lambda x,y:1 if x==y else 0,df["f_low"],df["llv_low"])

df["LLV_RSI1_20"] = map(lambda x:1 if x<20 else 0,df["llv_rsi1"])

# df["pct_chg_8"] = df["f_close"]/df["f_close"].shift(8)-1

#买入条件1
df["B_01"] = df["cross_rsi1_rsi2"]*df["LLV_RSI2_21"]

#买入条件2
df["B_02"] = df["cross_rsi2_20"]*df["LOW_LLV"]

#买入条件3
df["B_03"] = df["cross_rsi1_rsi2"]*df["LLV_RSI1_20"]

df["B"] = map(lambda x,y,z:1 if (x+y+z)>0 else 0 ,df["B_01"],df["B_02"],df["B_03"])

#卖出条件1
df["S_01"] = indictor.CROSS(df["H80"],df["rsi1"])

#卖出条件2
df["S_02"] = df["B"].shift(8)

# df["S_02"] = map(lambda x,y : 1 if x*y<0 else 0,df["B"].shift(8),df["pct_chg_8"])


df["S"] = map(lambda x,y:1 if x+y>0 else 0,df["S_01"],df["S_02"])

# df.to_excel("30m_result.xlsx",sheet_name="data")
df

    


util\indictor.py:43: RuntimeWarning: invalid value encountered in maximum
  return np.maximum(s1,s2)


,f_amount,f_close,f_date,f_high,f_low,f_open,f_preclose,f_seq,f_time,f_volume,...,cross_rsi2_20,LOW_LLV,LLV_RSI1_20,B_01,B_02,B_03,B,S_01,S_02,S
0,1.102054e+08,20.78,20170103,20.88,20.60,20.63,20.63,12,1000,5311472,...,0,0,0,0,0,0,0,0,NaN,0
1,5.633500e+07,20.71,20170103,20.81,20.67,20.79,20.78,13,1030,2716599,...,0,0,0,0,0,0,0,0,NaN,0
2,3.060239e+07,20.75,20170103,20.77,20.70,20.72,20.71,14,1100,1475796,...,1,0,0,0,0,0,0,0,NaN,0
3,5.279873e+07,20.72,20170103,20.75,20.71,20.75,20.75,15,1130,2546505,...,0,0,1,0,0,0,0,0,NaN,0
4,2.963818e+07,20.68,20170103,20.73,20.66,20.72,20.73,17,1330,1432380,...,0,1,0,0,0,0,0,0,NaN,0
5,3.242910e+07,20.68,20170103,20.70,20.67,20.68,20.68,18,1400,1567764,...,0,0,0,0,0,0,0,0,NaN,0
6,4.392430e+07,20.72,20170103,20.77,20.67,20.69,20.68,19,1430,2119076,...,0,0,0,1,0,0,1,0,NaN,0
7,8.691549e+07,20.73,20170103,20.73,20.68,20.73,20.72,20,1500,4197277,...,0,0,0,0,0,0,0,0,NaN,0
8,9.080559e+07,20.58,20170104,20.82,20.55,20.76,20.76,30372,1000,4395577,...,0,1,1,0,0,0,0,0,0.0,0
9,9.250333e+07,20.53,20170104,20.63,20.45,20.57,20.58,30373,1030,4507435,...,0,1,1,0,0,0,0,0,0.0,0


In [5]:
B = 0
S = 1

items = []
for i in range(len(df)):
    item = {}
    if df.at[i,"B"]==1:
        if S==1 and B==0:
            
            f_datetime = df.at[i,"f_datetime"]
            item["datetime"] = f_datetime
            item["index"] = i
            item["status"] = "B"
            item["close"] = df.at[i,"f_close"]
            item["date"] = df.at[i,"f_date"]
            
            print "买点时间：{}".format(f_datetime)
            B=1
            S=0
            items.append(item)
        else:
            print "索引{}的买点为重复买点".format(i)
    
    if df.at[i,"S"] == 1:
        if B==1 and S==0:
            
            f_datetime = df.at[i,"f_datetime"]
            item["datetime"] = f_datetime
            item["index"] = i
            item["status"] = "S"
            item["close"] = df.at[i,"f_close"]
            item["date"] = df.at[i,"f_date"]
            
            print "卖点时间：{}".format(df.at[i,"f_datetime"])
            B=0
            S=1
            items.append(item)
        else:
            print "索引{}的卖点为重复卖点".format(i)

def get_netValues(revenueValues):
    netv = 1
    netvs = []
    for rate in revenueValues:
        netv *= 1+rate
        netvs.append(netv)
    return netvs

#买卖点DataFrame
df_b_s = pd.DataFrame(items)    

df_b_s.to_excel("df_b_s.xlsx")

print '--------------'

#计算每次交易的收益率
df_b_s["pct_chg"] = df_b_s["close"].pct_change()  

#提取每次卖出点的数据
df_s = df_b_s.loc[1::2]

#交易获正收益的数据
df_up = df_s[df_s["pct_chg"]>0]

#计算统计值
stats = {}
stats["test_sum"] = len(df)         #测试周期
stats["trade_count"] = len(df_s)    #交易次数
stats["revenue_count"] = df_up["pct_chg"].count()    #盈利次数
stats["winrate"] = stats["revenue_count"]*1.0/stats["trade_count"]*100    #胜率
stats["rev_max"] = df_s["pct_chg"].max()*100     #单次交易最大盈利
stats["rev_min"] = df_s["pct_chg"].min()*100    #单次交易最大亏损
stats["rev_sum"] = (get_netValues(df_s["pct_chg"].tolist())[-1]-1)*100   #总盈利

pd.DataFrame([stats])

买点时间：201701031430
索引10的买点为重复买点
卖点时间：201701041430
索引18的卖点为重复卖点
买点时间：201701131000
卖点时间：201701131400
索引71的卖点为重复卖点
索引72的卖点为重复卖点
索引178的卖点为重复卖点
索引248的卖点为重复卖点
买点时间：201703031030
索引309的买点为重复买点
卖点时间：201703061030
索引317的卖点为重复卖点
索引333的卖点为重复卖点
索引364的卖点为重复卖点
索引474的卖点为重复卖点
买点时间：201704111430
卖点时间：201704121030
索引518的卖点为重复卖点
买点时间：201704271130
卖点时间：201704281130
买点时间：201704281500
卖点时间：201705021500
买点时间：201705041000
卖点时间：201705051000
买点时间：201705091100
卖点时间：201705101100
索引680的卖点为重复卖点
索引689的卖点为重复卖点
买点时间：201705221500
卖点时间：201705231500
索引759的卖点为重复卖点
索引770的卖点为重复卖点
索引834的卖点为重复卖点
索引902的卖点为重复卖点
索引907的卖点为重复卖点
索引910的卖点为重复卖点
索引918的卖点为重复卖点
索引920的卖点为重复卖点
买点时间：201707051130
卖点时间：201707061130
索引1011的卖点为重复卖点
索引1041的卖点为重复卖点
买点时间：201707261000
卖点时间：201707271000
索引1129的卖点为重复卖点
买点时间：201708081030
卖点时间：201708091030
买点时间：201708141500
卖点时间：201708151500
索引1226的卖点为重复卖点
索引1232的卖点为重复卖点
索引1242的卖点为重复卖点
索引1278的卖点为重复卖点
索引1322的卖点为重复卖点
索引1327的卖点为重复卖点
索引1329的卖点为重复卖点
索引1339的卖点为重复卖点
索引1373的卖点为重复卖点
索引1378的卖点为重复卖点
索引1383的卖点为重复卖点
索引1388的卖点为重复卖点
买点时

,rev_max,rev_min,rev_sum,revenue_count,test_sum,trade_count,winrate
0,4.349577,-4.145408,11.930054,24,3844,39,61.538462


In [38]:
import pandas as pd
df = pd.read_excel('result300.xlsx',sheet_name="data")
df.loc[:,["rev_max","rev_min","rev_sum","revenue_count","trade_count","winrate"]].describe()

,rev_max,rev_min,rev_sum,revenue_count,trade_count,winrate
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,6.982056,-15.306794,-2.456715,13.510000,21.350000,62.585343
std,2.515311,9.560488,26.912885,3.805897,4.323944,10.149147
min,2.161828,-56.130868,-69.530968,2.000000,5.000000,18.181818
25%,5.129444,-18.590867,-20.967086,12.000000,19.000000,56.630435
50%,6.796185,-12.712668,-4.553112,14.000000,22.000000,63.157895
75%,8.472070,-9.195743,13.642001,16.000000,24.000000,69.565217
max,16.138975,-1.081081,94.964349,23.000000,32.000000,88.000000


In [104]:
df = pd.read_excel("result_TEST1.xlsx",sheet_name="data")
df.loc[:,["rev_max","rev_min","rev_sum","revenue_count","trade_count","winrate"]].describe()

,rev_max,rev_min,rev_sum,revenue_count,trade_count,winrate
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,5.804903,-6.304620,3.694834,20.310000,41.110000,49.573939
std,2.741000,5.292556,16.669053,5.384286,8.441689,9.238204
min,1.111111,-51.585728,-59.045052,4.000000,11.000000,26.666667
25%,3.968817,-7.450748,-7.549258,17.000000,36.000000,42.767857
50%,5.208820,-5.308766,2.329880,20.000000,41.000000,49.135618
75%,6.979196,-3.713459,13.353174,24.000000,46.000000,56.097561
max,19.250000,-0.918485,52.835597,41.000000,63.000000,76.744186


In [34]:
df = pd.read_excel("df_cmp_bs.xlsx",sheet_name="data")
df_gp = df.groupby(["date","status"],as_index=False).count()
# df_gp = df.groupby(["date"],as_index=False).count()
# print df_gp[df_gp["status"]=="B"].mean()
# print df_gp[df_gp["status"]=="S"].mean()
# df_gp.describe()

# df_res = df_gp.loc[:,["date","status","count"]]
df_gp.to_excel("df_bs_count.xlsx")
df_gp

,date,status,close,datetime,index,stockcode,pct_chg
0,20170103,B,62,62,62,62,0
1,20170103,S,8,8,8,8,7
2,20170104,B,23,23,23,23,0
3,20170104,S,26,26,26,26,26
4,20170105,B,2,2,2,2,0
5,20170105,S,13,13,13,13,13
6,20170106,B,10,10,10,10,0
7,20170106,S,4,4,4,4,4
8,20170109,B,43,43,43,43,10
9,20170109,S,4,4,4,4,4


In [37]:
df = pd.read_excel("df_bs_count.xlsx",sheet_name="data")
df.loc[:,["B_COUNT","S_COUNT"]].describe()

,B_COUNT,S_COUNT
count,481.000000,481.000000
mean,13.667360,13.349272
std,15.845397,16.946566
min,0.000000,0.000000
25%,2.000000,4.000000
50%,8.000000,8.000000
75%,20.000000,17.000000
max,123.000000,194.000000


In [44]:
import pandas as pd
df = pd.read_excel('result300.xlsx',sheet_name="data")
df_60 = df[df["winrate"]>=60]
df_70 = df[df["winrate"]>=70]
df_70.loc[:,["rev_max","rev_min","rev_sum","revenue_count","trade_count","winrate"]].describe()

,rev_max,rev_min,rev_sum,revenue_count,trade_count,winrate
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
mean,7.448027,-14.708405,19.686629,16.657534,22.205479,74.927929
std,2.270228,10.254392,26.534783,3.448910,4.304258,4.384562
min,2.164009,-51.092896,-23.209478,5.000000,7.000000,70.000000
25%,5.772006,-19.089082,0.681767,15.000000,20.000000,71.428571
50%,7.675676,-12.307460,17.387070,17.000000,23.000000,73.913043
75%,8.888889,-8.481675,34.391712,19.000000,25.000000,76.923077
max,11.898017,-1.081081,94.964349,23.000000,29.000000,88.000000
